In [ ]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import time
import re
import os
import pandas as pd
entries = os.listdir("/volumes/Hayley's Drive/PycharmProjects/twilightvalefalls/")
dir = "/volumes/Hayley's Drive/PycharmProjects/twilightvalefalls/gravityfalls/"#%%

In [ ]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [ ]:
page = simple_get('https://gravityfalls.fandom.com/wiki/Transcripts#Season_1')

In [ ]:
html = BeautifulSoup(page, 'html.parser')
th = html.find_all('th')
match = re.findall(r"[0-9]*\.\s<a href=([\S]*)",str(th))

In [ ]:
base_page = 'https://gravityfalls.fandom.com'
epi_pages = list()
epi_pages2 = list()
for p in match:
    p = p.replace('"','')
    epi_pages.append(base_page + p + '/Transcript')
    epi_pages2.append(base_page + p)

In [ ]:
epis = dict()
for i,epi in enumerate(epi_pages):
    time.sleep(10)
    key = match[i]
    print(key, epi, time.localtime())
    gf = simple_get(epi)
    print(gf is None)
    epis[key] = gf

In [ ]:
dates = dict()
for i,epi in enumerate(epi_pages2):
    print(epi)
    time.sleep(10)
    key = match[i]
    print(key, epi, time.localtime())
    gf = simple_get(epi)
    print(gf is None)
    dates[key] = gf 

In [ ]:
keep_dates = dict()
for epi in dates:
    s = dates.get(epi)
    s = BeautifulSoup(s, 'html.parser')
    div = s.find_all('div', attrs={'data-source':'aired'})
    new_epi = re.sub(r'(\"\/wiki\/)([\w\.\%\!\-\,\:]*)(\")',r'\2',epi)
    air_date = re.findall(r'(">[\w \,]+<?)',str(div))[1]
    air_date = re.sub(r'(">)([\w \,]+)(<?)',r'\2',air_date)
    new_epi = new_epi.replace('_',' ').replace('%27',"'")
    keep_dates[new_epi] = air_date
    print(new_epi, air_date)

In [ ]:
ahref = dict()
epi_list = list()
for epi in epis:
    ahref_list = list()
    s = BeautifulSoup(epis.get(epi), 'html.parser')
    th = s.find_all(['th','tr'])
    th = re.sub(r"(\[<tr>\s)((<[\w\s=\"\-\:\;\/\.\?\#]*>\s?)*([\w\s\"\.\,]*))*",r"",str(th))
    th = th[re.search(r'<td>(\s)?<i>', th).start():th.find('\n</td></tr>, <th>\n</th>, <tr>\n<th ')]
    
    for i in re.finditer(r'<a(\sclass="mw-redirect")?\shref=[\"\/\w\-\#\,\%\(\)\:\.\!]*\stitle=[\"\s\w\-\,\'\(\)\:\.\!\?]*>', th):
        if(th.find(i[0])):
            ahref_list.append(i[0])
            th = th.replace(i[0],'')
    
    th = re.sub(r'</a>','',th)  
    th = re.sub(r'<th>[\w\-\s\,\'\&\;]*\s</th>, ','',th)
    th = re.sub(r'</td></tr>,\s<tr>\s<th>\s</th><td>','\n',th)
    th = re.sub(r'</td></tr>,\s<tr>\s<th>','\n',th)
    th = re.sub(r'\s</th><td>',': ',th)
    th = re.sub(r'<p>','',th)
    th = re.sub(r'</p>','',th)
    th = re.sub(r'<i>','',th)
    th = re.sub(r'</i>','',th)
    th = re.sub(r'<td>','',th)
    th = th.replace('<br>','').replace('<\/br>','').replace('&amp;','and')
    th = re.sub(r'</td></tr>, <th>','',th)
    
    ahref[re.sub(r'(\"\/wiki\/)([\w\.\%\!\-\,\:]*)(\")',r'\2',epi)] = ahref_list
    epi_list.append(re.sub(r'(\"\/wiki\/)([\w\.\%\!\-\,\:\!]*)(\")',r'\2',epi))
    
    f = open(dir + re.sub(r'(\"\/wiki\/)([\w\.\%\!\-\,\:]*)(\")',r'\2',epi)+'.txt',"w+", encoding="utf-8")
    f.write(th)
    f.close()

In [ ]:
list_story = list()
title_list = list()
air_dates = list()
source = ['gravity falls'] * 40
for ep in epi_list:
    print(dir + ep + '.txt')
    with open(dir + ep + '.txt', 'r', encoding="utf-8") as myfile:
        list_story.append(myfile.read().replace('\n',' '))
    title_list.append(ep.replace('_',' ').replace('%27',"'"))
    air_dates.append(keep_dates.get(ep.replace('_',' ').replace('%27',"'")))
list_story = [story.lower() for story in list_story]
gf_eps = pd.DataFrame({
    'source': source,
    'title': title_list,
    'date': air_dates,
    'text': list_story
}) 

In [ ]:
gf_eps.to_csv(path_or_buf=dir+'/gf_eps.csv', sep='|') 